<a href="https://colab.research.google.com/github/mjdileep/PII-Detect/blob/main/PII_Detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install seqeval
!pip install datasets
!pip install accelerate -U
!pip uninstall tensorflow -y
!pip install transformers[torch]

In [3]:
import multiprocessing

# Get the number of CPU cores
cpu_cores = multiprocessing.cpu_count()

In [4]:
import nltk
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Download the dataset

In [3]:
"""
!wget http://#######.us-east-2.compute.amazonaws.com:8000/mixtral-8x7b-v1_names_fixed.json
!wget http://#######.us-east-2.compute.amazonaws.com:8000/mpware_mixtral8x7b_v1.1-no-i-username_names_fixed.json
!wget http://#######.us-east-2.compute.amazonaws.com:8000/vw_moth_pii_dataset_fixed_names_fixed.json
!wget http://#######.us-east-2.compute.amazonaws.com:8000/vw_pj_moredata_dataset_fixed_names_fixed.json
!wget http://#######.us-east-2.compute.amazonaws.com:8000/illi_data_names_fixed.json
!wget http://#######.us-east-2.compute.amazonaws.com:8000/train.json
!wget http://#######.us-east-2.compute.amazonaws.com:8000/enable.txt
!cp *.json '/content/drive/MyDrive/AIML/PII Detect'
!cp *.txt '/content/drive/MyDrive/AIML/PII Detect'
"""

"\n!wget http://ec2-3-128-179-24.us-east-2.compute.amazonaws.com:8000/mixtral-8x7b-v1_names_fixed.json\n!wget http://ec2-3-128-179-24.us-east-2.compute.amazonaws.com:8000/mpware_mixtral8x7b_v1.1-no-i-username_names_fixed.json\n!wget http://ec2-3-128-179-24.us-east-2.compute.amazonaws.com:8000/vw_moth_pii_dataset_fixed_names_fixed.json\n!wget http://ec2-3-128-179-24.us-east-2.compute.amazonaws.com:8000/vw_pj_moredata_dataset_fixed_names_fixed.json\n!wget http://ec2-3-128-179-24.us-east-2.compute.amazonaws.com:8000/illi_data_names_fixed.json\n!wget http://ec2-3-128-179-24.us-east-2.compute.amazonaws.com:8000/train.json\n!wget http://ec2-3-128-179-24.us-east-2.compute.amazonaws.com:8000/enable.txt\n!cp *.json '/content/drive/MyDrive/AIML/PII Detect'\n!cp *.txt '/content/drive/MyDrive/AIML/PII Detect'\n"

In [5]:
enable_eng_wordlist = set()
with open("/content/drive/MyDrive/AIML/PII Detect/enable.txt", "r") as f:
    line = f.readline().strip()
    while line:
        enable_eng_wordlist.add(line)
        line = f.readline().strip()


In [6]:

TRAINING_MODEL_PATH = "microsoft/deberta-v3-large"  # your model path
TRAINING_MAX_LENGTH = 1280  # I use 1280 locally
base_location = "/content/drive/MyDrive/AIML/PII Detect/"
OUTPUT_DIR = base_location+'deberta_v3_large_2stepped'  # your output path
fill_label = "O"#ignore_label

In [7]:
import json
import os
import argparse
from itertools import chain
from functools import partial
import torch
import torch.nn as nn
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
from transformers import PreTrainedModel, PretrainedConfig
import datasets
from datasets import load_dataset, concatenate_datasets
from datasets import Dataset, features
import numpy as np
from tqdm import tqdm
import pandas as pd
import random

def set_random_seeds(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(42)
    np.random.seed(seed)
    random.seed(seed)
set_random_seeds(42)

from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TRAINING_MODEL_PATH)


In [9]:
all_labels = set(['B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT','B-PHONE_NUM','B-STREET_ADDRESS','B-URL_PERSONAL','B-USERNAME', 'I-ID_NUM','I-NAME_STUDENT','I-PHONE_NUM','I-STREET_ADDRESS','I-URL_PERSONAL','O'])
def is_exists(labels, targets):
    labels = set(labels)
    targets = set(targets)
    if labels.intersection(targets):
        return True
    return False


## Filter Dataset


In [10]:
targets = ['B-ID_NUM','B-PHONE_NUM','B-STREET_ADDRESS','B-URL_PERSONAL','B-USERNAME']

nicollas = json.load(open(base_location+"mixtral-8x7b-v1_names_fixed.json"))
l = len(nicollas)
nicollas_filtered = []
for doc in nicollas:
    if is_exists(doc["labels"], targets+['B-NAME_STUDENT']):
        nicollas_filtered.append(doc)
nicollas = nicollas_filtered
print(f"nicollas size({l}):", len(nicollas))


mpware = json.load(open(base_location+"mpware_mixtral8x7b_v1.1-no-i-username_names_fixed.json"))
mpware_filtered = []
for doc in mpware:
    if is_exists(doc["labels"], targets+['B-NAME_STUDENT']):
        mpware_filtered.append(doc)
mpware = mpware_filtered
print("mpware size:", len(mpware))

moth = json.load(open(base_location+"vw_moth_pii_dataset_fixed_names_fixed.json"))
moth_filtered = []
for doc in moth:
    if is_exists(doc["labels"], targets+['B-NAME_STUDENT']):
        moth_filtered.append(doc)
moth = moth_filtered
print("moth size:", len(moth))


pj = json.load(open(base_location+"vw_pj_moredata_dataset_fixed_names_fixed.json"))
pj_filtered = []
for doc in pj:
    if is_exists(doc["labels"], targets+['B-NAME_STUDENT']):
        pj_filtered.append(doc)
pj = pj_filtered
print("pj size:", len(pj))


filtered_docs = [1000185,1000988,1001022,1001093,1002098,1002141,1002211,1002246,1002398,1002411,1002599,1002660,1002809,1002854,1002926,1002970,1003005,1003149,1003211]
illi_data_names_fixed = []
for doc in json.load(open(base_location+"illi_data_names_fixed.json")):
    if doc["document"] not in filtered_docs:
        illi_data_names_fixed.append(doc)
illi_data_names_fixed_filtered = []
for doc in illi_data_names_fixed:
    if is_exists(doc["labels"], targets+['B-NAME_STUDENT']):
        illi_data_names_fixed_filtered.append(doc)
illi_data = illi_data_names_fixed_filtered
print("illi_data size:", len(illi_data))



nicollas size(2355): 2355
mpware size: 2322
moth size: 4423
pj size: 1999
illi_data size: 1623


In [10]:
def filter_tags(dataset, filters):
    filtered = []
    for doc in dataset:
        for label in doc["labels"]:
            if label[2:] in filters:
                filtered.append(doc)
                break
    print(f"Filters:{filters}:{len(filtered)}")
    return filtered

## Prepare Train Splits

In [11]:
STRATIFIED_COLUMNS = ['I-URL_PERSONAL',
                        'I-ID_NUM',
                        'B-STREET_ADDRESS',
                        'B-PHONE_NUM',
                        'B-USERNAME',
                        'I-PHONE_NUM',
                        'I-STREET_ADDRESS',
                        'B-EMAIL',
                        'B-ID_NUM',
                        'B-URL_PERSONAL',
                        'I-NAME_STUDENT',
                        'B-NAME_STUDENT',
                        'O'] # stratified labels with order
SPLITS = 5 #number of folds
SAVE_DIR = "." # folder to save the k folds json files


def create_stratified_folds(data, n_split=SPLITS, stratified_labels=STRATIFIED_COLUMNS):
    """
    Create stratified folds for cross-validation based on specified columns.

    Args:
        data (list): A list of dictionaries where each dictionary represents a sample.
        n_split (int, optional): Number of splits/folds to create. Defaults to SPLITS.
        stratified_labels (list, optional): List of labels to stratify the folds on. Defaults to STRATIFIED_COLUMNS.

    Returns:
        dict: A dictionary where keys represent the fold numbers (as strings) and values
              are lists containing the documents assigned to each fold.
    """
    processed = set()
    folds = {str(i):[] for i in range(n_split)}

    for label in stratified_labels:
        docs = []
        for sample in data:
            if label in sample['labels']:

                #don't process document more than once
                if sample['document'] not in processed:
                    docs.append(sample['document'])
                    processed.add(sample['document'])

        folds = distribute_docs(folds, docs)

    return folds

def create_document_fold_mapping(folds):
    """
    Creates a mapping of documents to folds.

    Args:
    - folds (dict): A dictionary containing fold IDs as keys and lists of documents as values.

    Returns:
    dict: A dictionary mapping each document to its respective fold.
    """
    document_fold_mapping = {}
    for fold, documents in folds.items():
        for document in documents:
            document_fold_mapping[document] = fold
    return document_fold_mapping


def distribute_docs(folds, doc_id):
    """
    Distributes document IDs equally among the folds in the given dictionary.

    Args:
    - folds (dict): A dictionary containing fold IDs as keys and empty lists as values.
    - doc_id (list): A list of document IDs to be distributed among the folds.

    Returns:
    dict: The modified folds dictionary with document IDs distributed equally among the folds.
    """
    combined_list = [item for sublist in folds.values() for item in sublist]

    num_folds = len(folds)
    chunk_size = len(doc_id) // num_folds
    remainder = len(doc_id) % num_folds

    for i in range(num_folds):
        start_idx = i * chunk_size + min(i, remainder)
        end_idx = (i + 1) * chunk_size + min(i + 1, remainder)
        folds[str(i)] += doc_id[start_idx:end_idx]

    new_combined_list = [item for sublist in folds.values() for item in sublist]

    return folds


def create_ksplits(data, document_fold_dict):
    """
    Split data samples into k splits based on pre-defined document fold dictionary.

    Args:
        data (list): A list of dictionaries where each dictionary represents a sample.
        document_fold_dict (dict): A dictionary where keys are document IDs and values
                                   represent the fold ID assigned to the document.

    Returns:
        dict: A dictionary where keys represent the split numbers (as strings) and values
              are lists containing the samples assigned to each split.
    """
    splits = {str(i):[] for i in range(SPLITS)}

    for sample in data:
        doc_id = sample['document']
        fold_id = document_fold_dict[doc_id]
        splits[fold_id].append(sample)

    return splits



In [12]:
train = json.load(open(base_location+"train.json"))
folds = create_stratified_folds(train)
document_fold_dict = create_document_fold_mapping(folds)
train_splits = create_ksplits(train, document_fold_dict)

suplimentary_splits = [
    nicollas,
    mpware,
    moth,
    pj,
    illi_data
]


In [13]:
def remove_B_only(dataset):
    filtered = []
    excluded = []
    for doc in dataset:
        is_student = False
        is_i = False
        for label in doc["labels"]:
            if "NAME_STUDENT" in label:
                is_student = True
            if label == "I-NAME_STUDENT":
                is_i = True
        if is_student and not is_i:
            excluded.append(doc)
            continue
        filtered.append(doc)
    return filtered, excluded

In [14]:
train_original = json.load(open(base_location+"train.json"))
def replace_test_docs(test_set):
    original = []
    for doc in test_set:
        for each in train_original:
            if doc["document"] == each["document"]:
                original.append(each)
                break
    return original

In [15]:
splits = []
no_of_splits = SPLITS

for i in range(no_of_splits):
    test_set = []
    train_set = []
    for j in range(no_of_splits):
        if i == j:
            test_set = train_splits[str(j)]
            train_set +=  suplimentary_splits[j]
        else:
            train_set += train_splits[str(j)]

    splits.append((train_set, test_set))

In [16]:
def is_english_word(word, debug=False):
    word = word.strip()
    if len(word)<2:
        return True

    if word.istitle():
        return False

    word = word.lower()

    lemma1 = lemmatizer.lemmatize(word, 'v')
    lemma2 = lemmatizer.lemmatize(word, 'n')
    lemma3 = lemmatizer.lemmatize(word, 'a')
    lemma4 = lemmatizer.lemmatize(word, 'r')
    lemma5 = lemmatizer.lemmatize(word, 's')

    lemmas = set([lemma1, lemma2, lemma3, lemma4, lemma5])

    for lemma in lemmas:
        if lemma in english_words:
            return True

    return False

english_words = set(words.words())
english_words=english_words.union(enable_eng_wordlist)
lemmatizer = WordNetLemmatizer()

In [17]:
all_labels = ['B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT','B-PHONE_NUM','B-STREET_ADDRESS','B-URL_PERSONAL','B-USERNAME',
              'I-ID_NUM','I-NAME_STUDENT','I-PHONE_NUM','I-STREET_ADDRESS','I-URL_PERSONAL','O']

In [18]:
all_labels = sorted(all_labels)
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}
print(id2label)


{0: 'B-EMAIL', 1: 'B-ID_NUM', 2: 'B-NAME_STUDENT', 3: 'B-PHONE_NUM', 4: 'B-STREET_ADDRESS', 5: 'B-URL_PERSONAL', 6: 'B-USERNAME', 7: 'I-ID_NUM', 8: 'I-NAME_STUDENT', 9: 'I-PHONE_NUM', 10: 'I-STREET_ADDRESS', 11: 'I-URL_PERSONAL', 12: 'O'}


In [19]:
target = ['B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT','B-PHONE_NUM','B-STREET_ADDRESS','B-URL_PERSONAL','B-USERNAME',
          'I-ID_NUM','I-NAME_STUDENT','I-PHONE_NUM','I-STREET_ADDRESS','I-URL_PERSONAL'
         ]

In [20]:
def tokenize(example, tokenizer, label2id):
    text = []

    # these are at the character level
    labels = []
    targets = []

    for t, l, ws in zip(example["tokens"], example["provided_labels"], example["trailing_whitespace"]):

        text.append(t)
        labels.extend([l]*len(t))

        if l in target:
            targets.append(1)
        else:
            targets.append(0)
        # if there is trailing whitespace
        if ws:
            text.append(" ")
            labels.append(fill_label)

    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=TRAINING_MAX_LENGTH)

    target_num = sum(targets)
    labels = np.array(labels)

    text = "".join(text)
    token_labels = []

    for start_idx, end_idx in tokenized.offset_mapping:

        # CLS token
        if start_idx == 0 and end_idx == 0:
            token_labels.append(label2id[fill_label])
            continue

        # case when token starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1

        token_labels.append(label2id[labels[start_idx]])

    length = len(tokenized.input_ids)

    return {
        **tokenized,
        "labels": token_labels,
        "length": length,
        "target_num": target_num,
        "group": 1 if target_num>0 else 0
    }

In [21]:
def relabel(doc):  # Calculate once outside the loop
    pred_labels = doc["predicted_labels"]
    for i, label in enumerate(doc["labels"]):
        try:
            if label < base_labels and label != pred_labels[i]:
                # Update the label only once per condition
                doc["labels"][i] = label2id['O-'+id2label[label]]
        except:
            print(d_idx, len(item["labels"]),len(pred_labels) )
    return doc


In [22]:
def get_ds(data):
    ds = Dataset.from_dict({
        "tokens": [x["tokens"] for x in data],
        "trailing_whitespace": [x["trailing_whitespace"] for x in data],
        "provided_labels": [x["labels"] for x in data],
    })
    ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id}, num_proc=cpu_cores)
    ds = ds.class_encode_column("group")
    return ds

In [23]:
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

def compute_metrics(p, all_labels):
    predictions, labels = p
    if type(predictions)==tuple:
        predictions=predictions[0]
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    ignore_indices = [-100]
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l not in ignore_indices]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l not in ignore_indices]
        for prediction, label in zip(predictions, labels)
    ]

    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)

    ignore_indices_insample = [label2id["O"], -100]
    true_predictions_insample = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l not in ignore_indices_insample]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels_insample = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l not in ignore_indices_insample]
        for prediction, label in zip(predictions, labels)
    ]

    recall_insample = recall_score(true_labels_insample, true_predictions_insample)
    precision_insample = precision_score(true_labels_insample, true_predictions_insample)
    f1_score_insample = (1 + 5*5) * recall_insample * precision_insample / (5*5*precision_insample + recall_insample)

    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score,
        'recall_insample': recall_insample,
        'precision_insample': precision_insample,
        'f1_insample': f1_score_insample
    }
    return results

In [24]:
# Function to get SpaCy tokens with their character positions
def get_token_positions(doc):
    start= 0
    positions = []
    i = 1
    text = ""
    for token, whitespace in zip(doc["tokens"], doc["trailing_whitespace"]):
        text += token + ["", " "][whitespace]
        position = (i, token, start, start+len(token))
        start=len(text)
        positions.append(position)
        i+=1
    return text, positions


def process_cluster(cluster, final_labels):
    if len(cluster) <= 2:
        return final_labels
    else:
        print(cluster)
    """
    if cluster[0][1] == 'B-URL_PERSONAL':
        fill_label = labels2id['I-URL_PERSONAL']
        for i in range(cluster[1][0], cluster[-1][0]):
            final_labels[i] = fill_label
        return final_labels
    el
    """
    return final_labels


def decode_labels(text, positions, predictions, offset_mapping, eng_tok=True, smooth=True):
    results = []
    final_labels = []
    last_position = 0
    assert len(predictions) == len(offset_mapping), "Offsets and predictions doesn't match!"
    for count, token, token_start, token_end in positions:
        labels = []
        text_chunk = ""
        if eng_tok and is_english_word(token): # Pre-filter with 0.994 recall
            labels.append(label2id["O"])
        else:
            for i in range(last_position, len(offset_mapping)):
                offset_start, offset_end = offset_mapping[i]
                label = predictions[i]
                if token_start <= offset_end and token_end > offset_start:
                    labels.append(label)
                    last_position = i
                    text_chunk += text[offset_start:offset_end]
                elif offset_start > token_end:
                    break
        prediction = label2id["O"]
        labels = np.array(labels)
        labels=labels[labels!=label2id["O"]]
        if labels.size!=0:
            prediction = mode(labels)
        results.append((count, token, text_chunk, token_start, token_end, prediction))
        final_labels.append(prediction)

    # Smooth out labels
    if smooth:
        for i in range(1, len(final_labels)-1):
            pre_label = id2label[final_labels[i-1]][1:]
            post_label = id2label[final_labels[i+1]][1:]
            stripped_token = results[i][1].strip("\n -,:[]';/?.><,()-_+*#|\\r\t'")
            if len(stripped_token)==0 and "STREET" in post_label and "STREET" in pre_label:
                final_labels[i] = final_labels[i+1]

    return results, torch.tensor(final_labels, dtype=int)

In [25]:
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
from statistics import mode

def evaluate_simple(predictions, labels, all_labels):

    # Remove ignored index (special tokens)
    ignore_indices = [-100]
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l not in ignore_indices]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l not in ignore_indices]
        for prediction, label in zip(predictions, labels)
    ]

    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)

    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

In [26]:
from collections import defaultdict
from typing import Dict


class PRFScore:
    """A precision / recall / F score."""

    def __init__(
        self,
        *,
        tp: int = 0,
        fp: int = 0,
        fn: int = 0,
    ) -> None:
        self.tp = tp
        self.fp = fp
        self.fn = fn

    def __len__(self) -> int:
        return self.tp + self.fp + self.fn

    def __iadd__(self, other):  # in-place add
        self.tp += other.tp
        self.fp += other.fp
        self.fn += other.fn
        return self

    def __add__(self, other):
        return PRFScore(
            tp=self.tp + other.tp, fp=self.fp + other.fp, fn=self.fn + other.fn
        )

    def score_set(self, cand: set, gold: set) -> None:
        self.tp += len(cand.intersection(gold))
        self.fp += len(cand - gold)
        self.fn += len(gold - cand)

    @property
    def precision(self) -> float:
        return self.tp / (self.tp + self.fp + 1e-100)

    @property
    def recall(self) -> float:
        return self.tp / (self.tp + self.fn + 1e-100)

    @property
    def f1(self) -> float:
        p = self.precision
        r = self.recall
        return 2 * ((p * r) / (p + r + 1e-100))

    @property
    def f5(self) -> float:
        beta = 5
        p = self.precision
        r = self.recall

        fbeta = (1+(beta**2))*p*r / ((beta**2)*p + r + 1e-100)
        return fbeta

    def to_dict(self) -> Dict[str, float]:
        return {"p": self.precision, "r": self.recall, "f5": self.f5}


def compute_metrics_eval(pred_df, gt_df):
    """
    Compute the LB metric (lb) and other auxiliary metrics
    """

    references = {(row.document, row.token, row.label) for row in gt_df.itertuples()}
    predictions = {(row.document, row.token, row.label) for row in pred_df.itertuples()}

    score_per_type = defaultdict(PRFScore)
    references = set(references)

    for ex in predictions:
        pred_type = ex[-1] # (document, token, label)
        if pred_type != 'O':
            pred_type = pred_type[2:] # avoid B- and I- prefix

        if pred_type not in score_per_type:
            score_per_type[pred_type] = PRFScore()

        if ex in references:
            score_per_type[pred_type].tp += 1
            references.remove(ex)
        else:
            score_per_type[pred_type].fp += 1

    for doc, tok, ref_type in references:
        if ref_type != 'O':
            ref_type = ref_type[2:] # avoid B- and I- prefix

        if ref_type not in score_per_type:
            score_per_type[ref_type] = PRFScore()
        score_per_type[ref_type].fn += 1

    totals = PRFScore()

    for prf in score_per_type.values():
        totals += prf

    return {
        "ents_p": totals.precision,
        "ents_r": totals.recall,
        "ents_f5": totals.f5,
        "ents_per_type": {k: v.to_dict() for k, v in score_per_type.items() if k!= 'O'},
    }

In [27]:
def pred_labels(data):
    final_preds = []
    model.eval()
    for example in tqdm(data):
        text, positions = get_token_positions(example)
        tokens = tokenizer(text, return_offsets_mapping=True,  return_tensors="pt")
        with torch.no_grad():
            rs = model(**{k:tokens[k].to(device) for k in ["input_ids", "token_type_ids", "attention_mask"]})
        offset_mapping = tokens["offset_mapping"][0].tolist()[1:-1]  # Get offset mappings
        predictions = (rs.logits[0,:,:].argmax(-1)).to(int).tolist()
        final_preds.append(predictions)
    return final_preds

In [28]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from transformers import TrainerCallback

class CustomCrossEntropyLossWithFN(nn.Module):
    def __init__(self, weights):
        super().__init__()
        self.weights = torch.tensor(weights).to('cuda')

    def forward(self, logits, targets):
        # Standard cross-entropy loss
        return F.cross_entropy(logits, targets, weight=self.weights)

# Create a custom Trainer class
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
         # Unpack inputs and get the model outputs
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Access the labels from inputs
        labels = inputs["labels"].view(-1).clone()

        # Determine the percentage of class 12 labels to drop
        drop_percentage = 0.30  # for example, 20%

        # Find indices where the labels are class 12
        class_12_indices = (labels == 12).nonzero(as_tuple=True)

        # Calculate how many labels to drop
        num_to_drop = int(drop_percentage * len(class_12_indices[0]))

        # Randomly choose indices of class 12 labels to set to ignore index (-100)
        drop_indices = class_12_indices[0][torch.randperm(len(class_12_indices[0]))[:num_to_drop]]
        labels[drop_indices] = -100  # -100 is commonly used as the ignore index in PyTorch

        # Compute the loss using the potentially modified labels
        custom_loss = CustomCrossEntropyLossWithFN(self.class_weights)(
            logits.view(-1, len(all_labels)), labels
        )

        # Return loss (and optionally outputs)
        return (custom_loss, outputs) if return_outputs else custom_loss

    def set_weights(self, weights):
        self.class_weights = weights

class CustomAfterEvalCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        optimizer.train()

class CustomBeforeTrainCallback(TrainerCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        optimizer.train()

class CustomBeforeEvalCallback(TrainerCallback):
    def on_evaluate_begin(self, args, state, control, **kwargs):
        optimizer.eval()


In [29]:
def get_trainer(ds_train, ds_test, optimizer, epochs=5, save_limit=1, acc=1, es=100, ss=100):
    args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        fp16=False,
        num_train_epochs=epochs,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=acc,
        report_to="none",
        evaluation_strategy="epoch",
        warmup_steps=es, ##
        learning_rate=2e-5, ##
        save_strategy="epoch",
        save_total_limit=save_limit,
        overwrite_output_dir=True,
        load_best_model_at_end=True,
        lr_scheduler_type='cosine', ##
        metric_for_best_model="f1",
        greater_is_better=True,
        seed=42,
        weight_decay = 0.01 ##
    )
    trainer = CustomTrainer(
        model=model,
        args=args,
        train_dataset=ds_train,
        eval_dataset=ds_test,
        data_collator=collator,
        tokenizer=tokenizer,
        #optimizers=(optimizer, None) ,
        compute_metrics=partial(compute_metrics, all_labels=all_labels),
        #callbacks=[CustomBeforeTrainCallback, CustomAfterEvalCallback, CustomBeforeEvalCallback]
    )
    return trainer

In [30]:
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

## Start Training

In [ ]:
device = 'cuda'
FREEZE_LAYERS_STEP2 = 6
skip_splits = []
for split, (train_split, test_split) in enumerate(splits):
    if split in skip_splits:
        continue

    model = AutoModelForTokenClassification.from_pretrained(
        TRAINING_MODEL_PATH,
        num_labels=len(all_labels),
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )


    if FREEZE_LAYERS_STEP2>0:
        print(f'Freezing {FREEZE_LAYERS_STEP2} layers.')
        for layer in model.deberta.encoder.layer[:FREEZE_LAYERS_STEP2]:
            for param in layer.parameters():
                param.requires_grad = False

    ds_train = get_ds(train_split)
    ds_test = get_ds(test_split)
    k=1
    acc=64/k
    ws = int(len(train_split)/(2*acc*k))
    optimizer = None

    trainer = get_trainer(ds_train.shuffle(seed=42), ds_test, optimizer, epochs=5, save_limit=1, acc=acc, es=ws, ss=ws)
    weights = [1.0]*(len(all_labels)-1)+[0.01]
    trainer.set_weights(weights)

    trainer.train()
    trainer.save_model(f"{base_location}pretrained_models/split{split}_deberta_v3_large_2stepped")
    del trainer, model, ds_train, ds_test
    torch.cuda.empty_cache()


## Load model and evaluate

In [ ]:
device = 'cuda'
split=1
model = AutoModelForTokenClassification.from_pretrained(f"{base_location}pretrained_models/split{split}_deberta_v3_large_2stepped")
model.to(device)
model.eval()

In [ ]:
device="cuda"
count=1
final_targets = []
final_preds = []
final_results = []
pred_dict = {
    "document":[],
    "token":[],
    "label":[]
}
target_dict = {
    "document":[],
    "token":[],
    "label":[]
}
for example in tqdm(splits[split][1]):
    text, positions = get_token_positions(example)
    tokens = tokenizer(text, return_offsets_mapping=True,  return_tensors="pt")
    with torch.no_grad():
        rs = model(**{k:tokens[k].to(device) for k in ["input_ids", "token_type_ids", "attention_mask"]})
    offset_mapping = tokens["offset_mapping"][0].tolist()[1:-1]  # Get offset mappings
    predictions = rs["logits"][0,1:-1,:].argmax(-1).to(int).tolist()
    _, final_labels = decode_labels(text, positions, predictions, offset_mapping)
    targets = torch.tensor([label2id[l] for l in example["labels"]], dtype=int)
    final_targets.append(targets)
    final_preds.append(final_labels)
    final_results.append(_)

    pred_dict["document"] +=[count]*len(final_labels)
    target_dict["document"] +=[count]*len(final_labels)

    pred_dict["token"] +=list(range(1, len(targets)+1))
    target_dict["token"] +=list(range(1, len(targets)+1))

    pred_dict["label"] +=final_labels
    target_dict["label"] +=targets
    count+=1
    del tokens
    torch.cuda.empty_cache()

pred_dict_filtered = {
    "document":[],
    "token":[],
    "label":[]
}

target_dict_filtered = {
    "document":[],
    "token":[],
    "label":[]
}
for i in range(len(pred_dict["label"])):
    pred_dict["label"][i] = id2label[pred_dict["label"][i].item()]
    if pred_dict["label"][i] != 'O':
        pred_dict_filtered["document"].append(pred_dict["document"][i] )
        pred_dict_filtered["token"].append(pred_dict["token"][i] )
        pred_dict_filtered["label"].append(pred_dict["label"][i] )

for i in range(len(target_dict["label"])):
    target_dict["label"][i] = id2label[target_dict["label"][i].item()]
    if target_dict["label"][i] != 'O':
        target_dict_filtered["document"].append(target_dict["document"][i] )
        target_dict_filtered["token"].append(target_dict["token"][i] )
        target_dict_filtered["label"].append(target_dict["label"][i] )

pred_df = pd.DataFrame(pred_dict_filtered)
gt_df = pd.DataFrame(target_dict_filtered)
eval_dict = compute_metrics_eval(pred_df, gt_df)
m = eval_dict['ents_f5']
print(f"LB = {round(m, 6)}")
eval_dict

In [ ]:
from google.colab import runtime
runtime.unassign()